# Analysis of DNA-MERFISH for CTP11

by Pu Zheng

2022.03.16

analysis for dataset:

\\10.245.74.158\Chromatin_NAS_4\20220316-P_brain_CTP11-12-13_from_0304

This data is DNA of uncleared MERFISH RNA:
    \\10.245.74.158\Chromatin_NAS_0\20220303-P_brain_M1_nonclear_adaptor


In [1]:
%run "..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

66284


# 0. fov parameters

In [2]:
reload(ia)
reload(classes)
reload(classes.batch_functions)
reload(classes.field_of_view)
reload(io_tools.load)
reload(get_img_info)
reload(visual_tools)
reload(ia.correction_tools)
reload(ia.correction_tools.alignment)
reload(ia.spot_tools.matching)
reload(ia.segmentation_tools.chromosome)
reload(ia.spot_tools.fitting)

<module 'ImageAnalysis3.spot_tools.fitting' from 'E:\\Users\\puzheng\\Documents\\ImageAnalysis3\\spot_tools\\fitting.py'>

In [11]:
fov_param = {'data_folder':r'\\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65',
             'save_folder':r'J:\Pu_Temp\20220419-CTP11-13_from_0415storm65',
             'experiment_type': 'DNA',
             'num_threads': 44,
             'correction_folder':r'\\10.245.74.158\Chromatin_NAS_0\Corrections\20210621-Corrections_lumencor_from_60_to_50',
             'shared_parameters':{
                     'single_im_size':[50,2048,2048],
                     'distance_zxy': [250, 108, 108],
                     'corr_channels':['750','647','561'],
                     'num_empty_frames': 0, 
                     'num_buffer_frames':0,
                     'corr_hot_pixel':True,
                     'corr_Z_shift':False,
                     'corr_bleed':True,
                     'min_num_seeds':5,
                     'max_num_seeds': 20000,
                     'spot_seeding_th': 800,
                     'normalize_intensity_local':False,
                     'normalize_intensity_background':False,
                     'corr_gaussian_highpass':True,
                 }, 
             }

# 1. Process Fov

## 1.1 process first round

In [12]:
_overwrite = False

_save_images = False

_fit_spots = True

sel_fov_ids = np.arange(0,80) # batch 1 in mendel
#sel_fov_ids = np.arange(80,160) # batch1+2 in mendel


for _fov_id in sel_fov_ids:
    if 'bad_fovs_ids' in locals() and  _fov_id in bad_fovs_ids:
        continue
    
    fov = classes.field_of_view.Field_of_View(fov_param, _fov_id=_fov_id,
                                              _color_info_kwargs={
                                                  '_color_filename':'Color_Usage',
                                              }, 
                                              _prioritize_saved_attrs=False,
                                              _save_info_to_file=False, # whether overwrite
                                              )
    # 2. Process image into candidate spots
    fov.parallel = False
    setattr(fov, "combo_ref_id", 0)
    fov._process_image_to_spots('combo', 
                                _sel_ids=[1,2,67],
                                _load_common_reference=True, _load_with_multiple=False,
                                _save_images=_save_images,
                                _warp_images=False, _fit_spots=True,
                                _overwrite_drift=False, _overwrite_image=_overwrite,
                                _overwrite_spot=_overwrite)
    # 3. unique
    #setattr(fov, "unique_ref_id", 0)
    #fov._process_image_to_spots('unique', 
    #                            _load_common_reference=True, _load_with_multiple=False,
    #                            _save_images=_save_images,
    #                            _warp_images=False, _fit_spots=True,
    #                            _overwrite_drift=False, _overwrite_image=_overwrite,
    #                            _overwrite_spot=_overwrite)
    # 4. Process DAPI image
    fov._load_dapi_image()

Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 68
- Number of field of views: 217
- Importing csv format color_usage file: \\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\Analysis\Color_Usage.csv
- header: ['Hyb', '750', '647', '561', '488', '405']
dict_keys(['H0C1', 'H1C2', 'H2C3', 'H3C4', 'H4C5', 'H5C6', 'H6C7', 'H7C8', 'H8C9', 'H9C10', 'H10C11', 'H11C12', 'H12C13', 'H13C14', 'H14C15', 'H15C16', 'H16C17', 'H17C18', 'H18C19', 'H19C20', 'H20C21', 'H21C22', 'H22C23', 'H23C24', 'H24C25', 'H25C26', 'H26C27', 'H27C28', 'H28C29', 'H29C30', 'H30C31', 'H31C32', 'H32C33', 'H33C34', 'H34C35', 'H35C36', 'H36C37', 'H37C38', 'H38C39', 'H39C40', 'H40C41', 'H41C42', 'H42C43', 'H43C44', 'H44C45', 'H45C46', 'H46C47', 'H47C48', 'H48C49', 'H49C50', 'H50C51', 'H51C52', 'H52C53', 'H53C54', 'H54C55', 'H55C56', 'H56C57', 'H57C58', 'H58C59', 'H59C60', 'H60C61', 'H61C62', 'H62C63', 'H63C64', 'H64C65', 'H65C66', 'H66C67', 'H67U2', 'H68U3', 'H69U4', 'H70U5', 'H71

KeyboardInterrupt: 

## 1.2 process relabel round

In [3]:
fov_param = {'data_folder':r'\\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65',
             'save_folder':r'J:\Pu_Temp\20220419-CTP11-13_from_0415storm65',
             'experiment_type': 'DNA',
             'num_threads': 44,
             'correction_folder':r'\\10.245.74.158\Chromatin_NAS_0\Corrections\20210621-Corrections_lumencor_from_60_to_50',
             'shared_parameters':{
                     'single_im_size':[50,2048,2048],
                     'distance_zxy': [250, 108, 108],
                     'corr_channels':['750','647','561'],
                     'num_empty_frames': 0, 
                     'num_buffer_frames':0,
                     'corr_hot_pixel':True,
                     'corr_Z_shift':False,
                     'corr_bleed':True,
                     'min_num_seeds':5,
                     'max_num_seeds': 20000,
                     'spot_seeding_th': 800,
                     'normalize_intensity_local':False,
                     'normalize_intensity_background':False,
                     'corr_gaussian_highpass':False,
                 }, 
             }

In [5]:
_overwrite = False

_save_images = False

_fit_spots = True

#sel_fov_ids = np.arange(0,80) # batch 1 in mendel
#sel_fov_ids = np.arange(80,200) # batch 2 in franklin
sel_fov_ids = np.arange(200,217) # batch 3 in mendel


for _fov_id in sel_fov_ids:
    if 'bad_fovs_ids' in locals() and  _fov_id in bad_fovs_ids:
        continue
    
    fov = classes.field_of_view.Field_of_View(fov_param, _fov_id=_fov_id,
                                              _color_info_kwargs={
                                                  '_color_filename':'Color_Usage',
                                              }, 
                                              _prioritize_saved_attrs=False,
                                              _save_info_to_file=False, # whether overwrite
                                              )
    # 2.1 Process image into candidate spots
    #fov.parallel = False
    #setattr(fov, "combo_ref_id", 0)
    #fov._process_image_to_spots('combo', 
    #                            _sel_ids=[1,2,67],
    #                            _load_common_reference=True, _load_with_multiple=False,
    #                            _save_images=_save_images,
    #                            _warp_images=False, _fit_spots=True,
    #                            _overwrite_drift=False, _overwrite_image=_overwrite,
    #                            _overwrite_spot=_overwrite)
    # 2.2 Process image into candidate spots
    fov.parallel = False
    try:
        fov._save_to_file('relabeled_combo')
    except:
        pass
    setattr(fov, "relabeled_combo_ref_id", 0)
    fov._process_image_to_spots('relabeled_combo', 
                                #_sel_ids=[1,2,67],
                                _load_common_reference=True, _load_with_multiple=False,
                                _save_images=_save_images,
                                _warp_images=False, _fit_spots=True,
                                _overwrite_drift=False, _overwrite_image=_overwrite,
                                _overwrite_spot=_overwrite)
    # 3. unique
    #setattr(fov, "unique_ref_id", 0)
    #fov._process_image_to_spots('unique', 
    #                            _load_common_reference=True, _load_with_multiple=False,
    #                            _save_images=_save_images,
    #                            _warp_images=False, _fit_spots=True,
    #                            _overwrite_drift=False, _overwrite_image=_overwrite,
    #                            _overwrite_spot=_overwrite)
    # 4. Process DAPI image
    fov._load_dapi_image()

Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 68
- Number of field of views: 217
- Importing csv format color_usage file: \\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\Analysis\Color_Usage.csv
- header: ['Hyb', '750', '647', '561', '488', '405']
dict_keys(['H0C1', 'H1C2', 'H2C3', 'H3C4', 'H4C5', 'H5C6', 'H6C7', 'H7C8', 'H8C9', 'H9C10', 'H10C11', 'H11C12', 'H12C13', 'H13C14', 'H14C15', 'H15C16', 'H16C17', 'H17C18', 'H18C19', 'H19C20', 'H20C21', 'H21C22', 'H22C23', 'H23C24', 'H24C25', 'H25C26', 'H26C27', 'H27C28', 'H28C29', 'H29C30', 'H30C31', 'H31C32', 'H32C33', 'H33C34', 'H34C35', 'H35C36', 'H36C37', 'H37C38', 'H38C39', 'H39C40', 'H40C41', 'H41C42', 'H42C43', 'H43C44', 'H44C45', 'H45C46', 'H46C47', 'H47C48', 'H48C49', 'H49C50', 'H50C51', 'H51C52', 'H52C53', 'H53C54', 'H54C55', 'H55C56', 'H56C57', 'H57C58', 'H58C59', 'H59C60', 'H60C61', 'H61C62', 'H62C63', 'H63C64', 'H64C65', 'H65C66', 'H66C67', 'H67C1_rep', 'H68U2', 'H69U3', 'H70U4', 

-- loading image from file:\\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\H67C1_rep\Conv_zscan_201.dax  in 9.481s
-- removing hot pixels for channels:['750', '647', '561', '488'] in 50.397s
-- bleedthrough correction for channels: ['750', '647', '561'] in 32.403s
-- illumination correction for channels: 750, 647, 561, 488, in 10.128s
-- apply bead_drift calculate for channel: 488
-- start aligning given source image to given reference image.
-- drift 0: [ -0.21 -17.98 -14.61] in 5.842s.
-- drift 1: [ -0.14 -18.07 -14.58] in 4.521s.
-- drift 2: [ -0.13 -18.1  -14.68] in 4.548s.
--- drifts for crops:[0 1 2] pass the thresold, exit cycle.
--- finish drift: [ -0.16 -18.05 -14.62] in 15.268s
-- generate translation function for chromatic correction for channels: ['750', '561'] and drift:[ -0.16 -18.05 -14.62] in 0.000s
-- finish correction in 120.079s
-- start fitting spots in channel:750, 7572 seeded with th=700, 7571 fitted in 458.819s.
-- start fitting spots i

-- loading image from file:\\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\H0C1\Conv_zscan_203.dax  in 8.164s
-- removing hot pixels for channels:['488'] in 10.938s
-- illumination correction for channels: 488, in 3.504s
-- -- generate translation function with drift:[0. 0. 0.] in 0.000s
-- finish correction in 23.754s
-- saving fov_info to file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_203.hdf5
++ base attributes saved:['relabeled_combo_ref_im'] in 11.816s.
-- checking relabeled_combo, region:[ 1  2 67] in 0.019s.
+ Start sequential pre-processing for 1 images
++ processed relabeled_combo ids: [ 1  2 67] + batch process image: \\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\H67C1_rep\Conv_zscan_203.dax for channels:['750', '647', '561']
- loading relabeled_combo info from file:Conv_zscan_203.hdf5 in 2.305s.
-- ['750', '647', '561'] images are required to process, [] images are loaded from save file: J:\Pu_Temp\20220

- 68 folders are found according to color-usage annotation.
+ loading fov_info from file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_205.hdf5
++ base attributes loaded:['combo_ref_im', 'dapi_im'] in 5.402s.
+ loading correction from file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_205.hdf5
++ load bleed correction profile directly from savefile.
++ load chromatic correction profile directly from savefile.
++ load chromatic_constants correction profile directly from savefile.
++ load illumination correction profile directly from savefile.
+ loading segmentation from file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_205.hdf5
++ base attributes loaded:[] in 0.010s.
-- saving relabeled_combo to file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_205.hdf5
--- relabeled_combo attributes updated:[] in 0.015s.
-- folders not selected, allow processing all 68 folders
+ load reference image from file:\\10.245.74.158\Chromatin_NAS_5\20220419-P_b

-- finish correction in 131.787s
-- start fitting spots in channel:750, 9304 seeded with th=700, 9304 fitted in 594.872s.
-- start fitting spots in channel:647, 3981 seeded with th=750, 3981 fitted in 190.605s.
-- start fitting spots in channel:561, 9666 seeded with th=500, 9666 fitted in 642.497s.
- writting relabeled_combo spots into file:J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_206.hdf5
-- updated spots for id:[1, 2, 67] in 0.032s
in 1564.29s.
directly return existing attribute.
Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 68
- Number of field of views: 217
- Importing csv format color_usage file: \\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\Analysis\Color_Usage.csv
- header: ['Hyb', '750', '647', '561', '488', '405']
dict_keys(['H0C1', 'H1C2', 'H2C3', 'H3C4', 'H4C5', 'H5C6', 'H6C7', 'H7C8', 'H8C9', 'H9C10', 'H10C11', 'H11C12', 'H12C13', 'H13C14', 'H14C15', 'H15C16', 'H16C17', 'H17C18', 'H18C19', 'H19C20', 'H20C21

...

++ base attributes loaded:['combo_ref_im', 'dapi_im'] in 4.680s.
+ loading correction from file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_210.hdf5
++ load bleed correction profile directly from savefile.
++ load chromatic correction profile directly from savefile.
++ load chromatic_constants correction profile directly from savefile.
++ load illumination correction profile directly from savefile.
+ loading segmentation from file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_210.hdf5
++ base attributes loaded:[] in 0.010s.
-- saving relabeled_combo to file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_210.hdf5
--- relabeled_combo attributes updated:[] in 0.015s.
-- folders not selected, allow processing all 68 folders
+ load reference image from file:\\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\H0C1\Conv_zscan_210.dax
- correct the whole fov for image: \\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_041

-- start fitting spots in channel:647, 3124 seeded with th=750, 3124 fitted in 316.397s.
-- start fitting spots in channel:561, 7764 seeded with th=500, 7764 fitted in 961.203s.
- writting relabeled_combo spots into file:J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_211.hdf5
-- updated spots for id:[1, 2, 67] in 0.058s
in 2297.50s.
directly return existing attribute.
Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 68
- Number of field of views: 217
- Importing csv format color_usage file: \\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\Analysis\Color_Usage.csv
- header: ['Hyb', '750', '647', '561', '488', '405']
dict_keys(['H0C1', 'H1C2', 'H2C3', 'H3C4', 'H4C5', 'H5C6', 'H6C7', 'H7C8', 'H8C9', 'H9C10', 'H10C11', 'H11C12', 'H12C13', 'H13C14', 'H14C15', 'H15C16', 'H16C17', 'H17C18', 'H18C19', 'H19C20', 'H20C21', 'H21C22', 'H22C23', 'H23C24', 'H24C25', 'H25C26', 'H26C27', 'H27C28', 'H28C29', 'H29C30', 'H30C31', 'H31C32', 'H32C33',

- loading relabeled_combo info from file:Conv_zscan_213.hdf5 in 1.527s.
-- ['750', '647', '561'] images are required to process, [] images are loaded from save file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_213.hdf5
- correct the whole fov for image: \\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\H67C1_rep\Conv_zscan_213.dax
-- loading image from file:\\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\H67C1_rep\Conv_zscan_213.dax  in 12.282s
-- removing hot pixels for channels:['750', '647', '561', '488'] in 89.119s
-- bleedthrough correction for channels: ['750', '647', '561'] in 62.810s
-- illumination correction for channels: 750, 647, 561, 488, in 21.671s
-- apply bead_drift calculate for channel: 488
-- start aligning given source image to given reference image.
-- drift 0: [ -0.14 -18.9   -5.39] in 11.100s.
-- drift 1: [ -0.07 -18.97  -5.26] in 11.751s.
-- drift 2: [ -0.19 -18.96  -5.27] in 12.630s.
--- drifts fo

++ load illumination correction profile directly from savefile.
+ loading segmentation from file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_215.hdf5
++ base attributes loaded:[] in 0.015s.
-- saving relabeled_combo to file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_215.hdf5
--- relabeled_combo attributes updated:[] in 0.027s.
-- folders not selected, allow processing all 68 folders
+ load reference image from file:\\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\H0C1\Conv_zscan_215.dax
- correct the whole fov for image: \\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\H0C1\Conv_zscan_215.dax
-- loading illumination correction profile from file:
	 488 illumination_correction_488_2048x2048.npy
-- loading image from file:\\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\H0C1\Conv_zscan_215.dax  in 10.497s
-- removing hot pixels for channels:['488'] in 22.659s
-- illumination correcti

in 895.42s.
directly return existing attribute.


# redo the rest

In [3]:
fov_param = {'data_folder':r'\\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65',
             'save_folder':r'J:\Pu_Temp\20220419-CTP11-13_from_0415storm65',
             'experiment_type': 'DNA',
             'num_threads': 44,
             'correction_folder':r'\\10.245.74.158\Chromatin_NAS_0\Corrections\20210621-Corrections_lumencor_from_60_to_50',
             'shared_parameters':{
                     'single_im_size':[50,2048,2048],
                     'distance_zxy': [250, 108, 108],
                     'corr_channels':['750','647','561'],
                     'num_empty_frames': 0, 
                     'num_buffer_frames':0,
                     'corr_hot_pixel':True,
                     'corr_Z_shift':False,
                     'corr_bleed':True,
                     'min_num_seeds':5,
                     'max_num_seeds': 20000,
                     'spot_seeding_th': 800,
                     'normalize_intensity_local':False,
                     'normalize_intensity_background':False,
                     'corr_gaussian_highpass':False,
                 }, 
             }

In [ ]:
_overwrite = False

_save_images = False

_fit_spots = True

#sel_fov_ids = np.arange(0,80) # batch 1 in mendel
#sel_fov_ids = np.arange(80,160) # batch1+2 in mendel


for _fov_id in sel_fov_ids:
    if 'bad_fovs_ids' in locals() and  _fov_id in bad_fovs_ids:
        continue
    
    fov = classes.field_of_view.Field_of_View(fov_param, _fov_id=_fov_id,
                                              _color_info_kwargs={
                                                  '_color_filename':'Color_Usage',
                                              }, 
                                              _prioritize_saved_attrs=False,
                                              _save_info_to_file=False, # whether overwrite
                                              )
    # 2. Process image into candidate spots
    fov.parallel = True
    setattr(fov, "combo_ref_id", 0)
    fov._process_image_to_spots('combo', 
                                _load_common_reference=True, _load_with_multiple=False,
                                _save_images=_save_images,
                                _warp_images=False, _fit_spots=True,
                                _overwrite_drift=False, _overwrite_image=_overwrite,
                                _overwrite_spot=_overwrite)
    # 3. unique
    #setattr(fov, "unique_ref_id", 0)
    #fov._process_image_to_spots('unique', 
    #                            _load_common_reference=True, _load_with_multiple=False,
    #                            _save_images=_save_images,
    #                            _warp_images=False, _fit_spots=True,
    #                            _overwrite_drift=False, _overwrite_image=_overwrite,
    #                            _overwrite_spot=_overwrite)
    # 4. Process DAPI image
    #fov._load_dapi_image()

Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 68
- Number of field of views: 217
- Importing csv format color_usage file: \\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\Analysis\Color_Usage.csv
- header: ['Hyb', '750', '647', '561', '488', '405']
dict_keys(['H0C1', 'H1C2', 'H2C3', 'H3C4', 'H4C5', 'H5C6', 'H6C7', 'H7C8', 'H8C9', 'H9C10', 'H10C11', 'H11C12', 'H12C13', 'H13C14', 'H14C15', 'H15C16', 'H16C17', 'H17C18', 'H18C19', 'H19C20', 'H20C21', 'H21C22', 'H22C23', 'H23C24', 'H24C25', 'H25C26', 'H26C27', 'H27C28', 'H28C29', 'H29C30', 'H30C31', 'H31C32', 'H32C33', 'H33C34', 'H34C35', 'H35C36', 'H36C37', 'H37C38', 'H38C39', 'H39C40', 'H40C41', 'H41C42', 'H42C43', 'H43C44', 'H44C45', 'H45C46', 'H46C47', 'H47C48', 'H48C49', 'H49C50', 'H50C51', 'H51C52', 'H52C53', 'H53C54', 'H54C55', 'H55C56', 'H56C57', 'H57C58', 'H58C59', 'H59C60', 'H60C61', 'H61C62', 'H62C63', 'H63C64', 'H64C65', 'H65C66', 'H66C67', 'H67U2', 'H68U3', 'H69U4', 'H70U5', 'H71

-- checking combo, region:[41 42 87] in 0.049s.
-- checking combo, region:[43 44 88] in 0.050s.
-- checking combo, region:[45 46 89] in 0.048s.
-- checking combo, region:[47 48 90] in 0.049s.
-- checking combo, region:[49 50 91] in 0.048s.
-- checking combo, region:[51 52 92] in 0.069s.
-- checking combo, region:[53 54 93] in 0.067s.
-- checking combo, region:[55 56 94] in 0.052s.
-- checking combo, region:[57 58 95] in 0.052s.
-- checking combo, region:[59 60 96] in 0.052s.
-- checking combo, region:[61 62 97] in 0.051s.
-- checking combo, region:[63 64 98] in 0.052s.
-- checking combo, region:[65 66 99] in 0.052s.
-- checking combo, region:[101 102 165] in 0.070s.
-- checking combo, region:[151 152 190] in 0.072s.
-- checking combo, region:[153 154 191] in 0.049s.
-- checking combo, region:[155 156 192] in 0.048s.
-- checking combo, region:[157 158 193] in 0.049s.
-- checking combo, region:[159 160 194] in 0.049s.
-- checking combo, region:[161 162 195] in 0.052s.
-- checking combo, 

 186 187 188 189 195] , finish in 2303.51s.
Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 68
- Number of field of views: 217
- Importing csv format color_usage file: \\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\Analysis\Color_Usage.csv
- header: ['Hyb', '750', '647', '561', '488', '405']
dict_keys(['H0C1', 'H1C2', 'H2C3', 'H3C4', 'H4C5', 'H5C6', 'H6C7', 'H7C8', 'H8C9', 'H9C10', 'H10C11', 'H11C12', 'H12C13', 'H13C14', 'H14C15', 'H15C16', 'H16C17', 'H17C18', 'H18C19', 'H19C20', 'H20C21', 'H21C22', 'H22C23', 'H23C24', 'H24C25', 'H25C26', 'H26C27', 'H27C28', 'H28C29', 'H29C30', 'H30C31', 'H31C32', 'H32C33', 'H33C34', 'H34C35', 'H35C36', 'H36C37', 'H37C38', 'H38C39', 'H39C40', 'H40C41', 'H41C42', 'H42C43', 'H43C44', 'H44C45', 'H45C46', 'H46C47', 'H47C48', 'H48C49', 'H49C50', 'H50C51', 'H51C52', 'H52C53', 'H53C54', 'H54C55', 'H55C56', 'H56C57', 'H57C58', 'H58C59', 'H59C60', 'H60C61', 'H61C62', 'H62C63', 'H63C64', 'H64C65', 'H65C66', 'H66C6

-- checking combo, region:[37 38 85] in 0.043s.
-- checking combo, region:[39 40 86] in 0.055s.
-- checking combo, region:[41 42 87] in 0.045s.
-- checking combo, region:[43 44 88] in 0.043s.
-- checking combo, region:[45 46 89] in 0.045s.
-- checking combo, region:[47 48 90] in 0.044s.
-- checking combo, region:[49 50 91] in 0.044s.
-- checking combo, region:[51 52 92] in 0.062s.
-- checking combo, region:[53 54 93] in 0.059s.
-- checking combo, region:[55 56 94] in 0.046s.
-- checking combo, region:[57 58 95] in 0.046s.
-- checking combo, region:[59 60 96] in 0.046s.
-- checking combo, region:[61 62 97] in 0.046s.
-- checking combo, region:[63 64 98] in 0.047s.
-- checking combo, region:[65 66 99] in 0.045s.
-- checking combo, region:[101 102 165] in 0.062s.
-- checking combo, region:[151 152 190] in 0.063s.
-- checking combo, region:[153 154 191] in 0.044s.
-- checking combo, region:[155 156 192] in 0.045s.
-- checking combo, region:[157 158 193] in 0.048s.
-- checking combo, region

 188 189] , finish in 2081.90s.
Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 68
- Number of field of views: 217
- Importing csv format color_usage file: \\10.245.74.158\Chromatin_NAS_5\20220419-P_brain_CTP11-13_from_0415storm65\Analysis\Color_Usage.csv
- header: ['Hyb', '750', '647', '561', '488', '405']
dict_keys(['H0C1', 'H1C2', 'H2C3', 'H3C4', 'H4C5', 'H5C6', 'H6C7', 'H7C8', 'H8C9', 'H9C10', 'H10C11', 'H11C12', 'H12C13', 'H13C14', 'H14C15', 'H15C16', 'H16C17', 'H17C18', 'H18C19', 'H19C20', 'H20C21', 'H21C22', 'H22C23', 'H23C24', 'H24C25', 'H25C26', 'H26C27', 'H27C28', 'H28C29', 'H29C30', 'H30C31', 'H31C32', 'H32C33', 'H33C34', 'H34C35', 'H35C36', 'H36C37', 'H37C38', 'H38C39', 'H39C40', 'H40C41', 'H41C42', 'H42C43', 'H43C44', 'H44C45', 'H45C46', 'H46C47', 'H47C48', 'H48C49', 'H49C50', 'H50C51', 'H51C52', 'H52C53', 'H53C54', 'H54C55', 'H55C56', 'H56C57', 'H57C58', 'H58C59', 'H59C60', 'H60C61', 'H61C62', 'H62C63', 'H63C64', 'H64C65', 'H65C66', 'H66C67', 'H67C1_r

...

- 68 folders are found according to color-usage annotation.
+ loading fov_info from file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_050.hdf5
++ base attributes loaded:['combo_ref_im', 'dapi_im'] in 6.382s.
+ loading correction from file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_050.hdf5
++ load bleed correction profile directly from savefile.
++ load chromatic correction profile directly from savefile.
++ load chromatic_constants correction profile directly from savefile.
++ load illumination correction profile directly from savefile.
+ loading segmentation from file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_050.hdf5
++ base attributes loaded:[] in 0.012s.
-- folders not selected, allow processing all 68 folders
-- checking combo, region:[ 1  2 67] in 0.053s.
-- checking combo, region:[ 3  4 68] in 0.030s.
-- checking combo, region:[ 5  6 69] in 0.034s.
-- checking combo, region:[ 7  8 70] in 0.039s.
-- checking combo, region:[ 9 10 71] in

-- checking combo, region:[113 114 171] in 0.022s.
-- checking combo, region:[115 116 172] in 0.022s.
-- checking combo, region:[117 118 173] in 0.022s.
-- checking combo, region:[119 120 174] in 0.026s.
-- checking combo, region:[121 122] in 0.016s.
-- checking combo, region:[123 124] in 0.016s.
-- checking combo, region:[125 126 177] in 0.022s.
-- checking combo, region:[127 128 178] in 0.022s.
-- checking combo, region:[129 130] in 0.016s.
-- checking combo, region:[131 132 180] in 0.020s.
-- checking combo, region:[133 134 181] in 0.020s.
-- checking combo, region:[135 136 182] in 0.020s.
-- checking combo, region:[137 138 183] in 0.020s.
-- checking combo, region:[139 140 184] in 0.020s.
-- checking combo, region:[141 142 185] in 0.020s.
-- checking combo, region:[143 144 186] in 0.020s.
-- checking combo, region:[145 146 187] in 0.022s.
-- checking combo, region:[147 148 188] in 0.022s.
-- checking combo, region:[149 150 189] in 0.022s.
+ Start multi-processing of pre-processing 

++ load illumination correction profile directly from savefile.
+ loading segmentation from file: J:\Pu_Temp\20220419-CTP11-13_from_0415storm65\Conv_zscan_053.hdf5
++ base attributes loaded:[] in 0.013s.
-- folders not selected, allow processing all 68 folders
-- checking combo, region:[ 1  2 67] in 0.045s.
-- checking combo, region:[ 3  4 68] in 0.031s.
-- checking combo, region:[ 5  6 69] in 0.030s.
-- checking combo, region:[ 7  8 70] in 0.029s.
-- checking combo, region:[ 9 10 71] in 0.029s.
-- checking combo, region:[11 12 72] in 0.029s.
-- checking combo, region:[13 14 73] in 0.032s.
-- checking combo, region:[15 16 74] in 0.030s.
-- checking combo, region:[17 18 75] in 0.031s.
-- checking combo, region:[19 20 76] in 0.029s.
-- checking combo, region:[21 22 77] in 0.031s.
-- checking combo, region:[23 24 78] in 0.029s.
-- checking combo, region:[25 26 79] in 0.034s.
-- checking combo, region:[27 28 80] in 0.032s.
-- checking combo, region:[29 30 81] in 0.032s.
-- checking combo, 

-- checking combo, region:[127 128 178] in 0.046s.
-- checking combo, region:[129 130] in 0.035s.
-- checking combo, region:[131 132 180] in 0.037s.
-- checking combo, region:[133 134 181] in 0.029s.
-- checking combo, region:[135 136 182] in 0.029s.
-- checking combo, region:[137 138 183] in 0.030s.
-- checking combo, region:[139 140 184] in 0.029s.
-- checking combo, region:[141 142 185] in 0.029s.
-- checking combo, region:[143 144 186] in 0.030s.
-- checking combo, region:[145 146 187] in 0.042s.
-- checking combo, region:[147 148 188] in 0.042s.
-- checking combo, region:[149 150 189] in 0.044s.
+ Start multi-processing of pre-processing for 13 images with 44 threads
++ processing combo ids: [125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142
 143 144 145 146 147 148 149 150 177 178 180 181 182 183 184 185 186 187
 188 189] , finish in 1323.30s.
Get Folder Names: (ia.get_img_info.get_folders)
- Number of folders: 68
- Number of field of views: 217
- Importing 

-- checking combo, region:[ 5  6 69] in 0.064s.
-- checking combo, region:[ 7  8 70] in 0.064s.
-- checking combo, region:[ 9 10 71] in 0.064s.
-- checking combo, region:[11 12 72] in 0.062s.
-- checking combo, region:[13 14 73] in 0.071s.
-- checking combo, region:[15 16 74] in 0.053s.
-- checking combo, region:[17 18 75] in 0.049s.
-- checking combo, region:[19 20 76] in 0.050s.
-- checking combo, region:[21 22 77] in 0.051s.
-- checking combo, region:[23 24 78] in 0.051s.
-- checking combo, region:[25 26 79] in 0.064s.
-- checking combo, region:[27 28 80] in 0.055s.
-- checking combo, region:[29 30 81] in 0.044s.
-- checking combo, region:[31 32 82] in 0.044s.
-- checking combo, region:[33 34 83] in 0.043s.
-- checking combo, region:[35 36 84] in 0.043s.
-- checking combo, region:[37 38 85] in 0.043s.
-- checking combo, region:[39 40 86] in 0.055s.
-- checking combo, region:[41 42 87] in 0.044s.
-- checking combo, region:[43 44 88] in 0.045s.
-- checking combo, region:[45 46 89] in 